In [1]:
import pymysql
import pandas as pd

In [2]:
myconnection=pymysql.connect(host='',user='',password='')

In [5]:
import pandas as pd
df=pd.read_csv("sale.csv")

In [155]:
df.fillna("NULL",inplace=True)

In [67]:
df["Delivery Date"]=df["Delivery Date"].str.replace(r'[Null]', '', regex=True)

In [157]:
df

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,2016-01-01,NULL,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,NULL,1216913,43,632,3,USD
62880,2243031,1,2021-02-20,2021-02-24,511229,0,98,4,EUR
62881,2243032,1,2021-02-20,2021-02-23,331277,0,1613,2,CAD
62882,2243032,2,2021-02-20,2021-02-23,331277,0,1717,2,CAD


In [13]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Delivery Date'] = pd.to_datetime(df['Delivery Date'])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order Number   62884 non-null  int64         
 1   Line Item      62884 non-null  int64         
 2   Order Date     62884 non-null  datetime64[ns]
 3   Delivery Date  13165 non-null  datetime64[ns]
 4   CustomerKey    62884 non-null  int64         
 5   StoreKey       62884 non-null  int64         
 6   ProductKey     62884 non-null  int64         
 7   Quantity       62884 non-null  int64         
 8   Currency Code  62884 non-null  object        
dtypes: datetime64[ns](2), int64(6), object(1)
memory usage: 4.3+ MB


In [17]:
a=",".join(f"{i} {j}"
for i,j in zip(df.columns,df.dtypes)).replace("object","text").replace("float64","float").replace("int64","int").replace("datetime64[ns]","date")

In [19]:
a

'Order Number int,Line Item int,Order Date date,Delivery Date date,CustomerKey int,StoreKey int,ProductKey int,Quantity int,Currency Code text'

In [163]:
sql="insert into dataspark.sales (Order_number,Line_item,Order_date,Delivery_date,Customer_key,Store_key,Product_key,Quantity,Currency_code) values"
for i in range(10):
    print(f"{sql} {tuple(df.iloc[i])}")

insert into dataspark.sales (Order_number,Line_item,Order_date,Delivery_date,Customer_key,Store_key,Product_key,Quantity,Currency_code) values (366000, 1, '2016-01-01', 'NULL', 265598, 10, 1304, 1, 'CAD')
insert into dataspark.sales (Order_number,Line_item,Order_date,Delivery_date,Customer_key,Store_key,Product_key,Quantity,Currency_code) values (366001, 1, '2016-01-01', '2016-01-13', 1269051, 0, 1048, 2, 'USD')
insert into dataspark.sales (Order_number,Line_item,Order_date,Delivery_date,Customer_key,Store_key,Product_key,Quantity,Currency_code) values (366001, 2, '2016-01-01', '2016-01-13', 1269051, 0, 2007, 1, 'USD')
insert into dataspark.sales (Order_number,Line_item,Order_date,Delivery_date,Customer_key,Store_key,Product_key,Quantity,Currency_code) values (366002, 1, '2016-01-01', '2016-01-12', 266019, 0, 1106, 7, 'CAD')
insert into dataspark.sales (Order_number,Line_item,Order_date,Delivery_date,Customer_key,Store_key,Product_key,Quantity,Currency_code) values (366002, 2, '2016-01

In [ ]:
myconnection.cursor().execute(f"create table dataspark.sales (Order_number int,Line_item int,Order_date date,Delivery_date date,Customer_key int,FOREIGN KEY (Customer_key) REFERENCES customers(Customer_key),Store_key int,FOREIGN KEY (Store_key) REFERENCES stores(Store_key),Product_key int,FOREIGN KEY (Product_key) REFERENCES products(Product_key),Quantity int,Currency_code text)")
myconnection.commit()

In [165]:
sql="insert into dataspark.sales (Order_number,Line_item,Order_date,Delivery_date,Customer_key,Store_key,Product_key,Quantity,Currency_code) values"
for i in range(len(df)):
    myconnection.cursor().execute(f"{sql} {tuple(df.iloc[i])}")
    myconnection.commit()
    